Import BeautifulSoup Library and initializing website URL to scrap data

In [29]:
import requests

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [30]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())


 Create a parse tree for parsed pages that can be used to extract data from HTML, which is useful for web scraping

In [31]:
My_table = soup.find('table',{'class':'wikitable sortable'})

In [10]:
#My_table

Use tableDataText function to parse a html segment started with table tags 

In [32]:
def tableDataText(table):       
    rows = []
    trs = table.find_all('tr')
    headerow = [td.get_text(strip=True) for td in trs[0].find_all('th')] # header row
    if headerow: # if there is a header row include first
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs: # for every table row
        rows.append([td.get_text(strip=True) for td in tr.find_all('td')]) # data row
    return rows

In [33]:
list_table = tableDataText(My_table)
list_table[:5]

[['Postal Code', 'Borough', 'Neighborhood'],
 ['M1A', 'Not assigned', ''],
 ['M2A', 'Not assigned', ''],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village']]

Transform data into Panda dataframe

In [34]:
import pandas as pd
dftable = pd.DataFrame(list_table[1:], columns=list_table[0])
dftable.head(4)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village


                                 Eliminate data where 'Borough' is Not assigned

In [35]:
dftable.drop(dftable[dftable.Borough =='Not assigned'].index, inplace=True)

In [36]:
dftable.head(5)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


As Geocoder Python package was not working consistantly and google is charging for Google Maps Geocoding API ,Data loaded from csv file 

In [37]:
df_latlong = pd.read_csv('http://cocl.us/Geospatial_data')

In [38]:
df_latlong.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Two dataframes are merged to generate final result

In [39]:
df_results = pd.merge(left=dftable, right=df_latlong, left_on='Postal Code', right_on='Postal Code')

In [40]:
df_results.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
